## 01. Pemahaman Domain (Domain Understanding)

---



In [213]:
import pandas as pd
import re
import nltk

In [214]:
data1 = pd.read_csv('crawling_tweet/timnas_1.csv')
data2 = pd.read_csv('crawling_tweet/timnas_2.csv')
data3 = pd.read_csv('crawling_tweet/timnas_3.csv')
data4 = pd.read_csv('crawling_tweet/timnas_4.csv')

data = pd.concat([data1, data2, data3, data4], ignore_index=True)

In [215]:
data 

,conversation_id_str,created_at,favorite_count,full_text,id_str,image_url,in_reply_to_screen_name,lang,location,quote_count,reply_count,retweet_count,tweet_url,user_id_str,username
0,1800679330726715550,Tue Jun 11 23:59:42 +0000 2024,1,TIMNAS PRIA DAN WANITA MENANG + ERIK TEN HAG STAY,1800679330726715550,NaN,NaN,in,Indonesia,0,0,0,https://x.com/tigapuluhtigax/status/1800679330...,1188470356668895234,tigapuluhtigax
1,1800679305925800015,Tue Jun 11 23:59:36 +0000 2024,1,ngerasa ga sih sekarang timnas kalo main pakek...,1800679305925800015,NaN,NaN,in,"Jombang, Indonesia",0,0,0,https://x.com/sotongism/status/180067930592580...,961056397063409664,sotongism
2,1800679288326476196,Tue Jun 11 23:59:31 +0000 2024,0,2 Sejarah Dicetak Timnas Indonesia Usai Ganyan...,1800679288326476196,NaN,NaN,in,NaN,0,0,0,https://x.com/peanutkacang123/status/180067928...,1464596330421100548,peanutkacang123
3,1800589420766355730,Tue Jun 11 23:59:28 +0000 2024,0,@womensfootie_id Tinggal tmbah pemain diaspora...,1800679273185047018,NaN,womensfootie_id,in,NaN,0,0,0,https://x.com/PKananku1927/status/180067927318...,1772880651248939008,PKananku1927
4,1800679270857286043,Tue Jun 11 23:59:27 +0000 2024,0,Timnas Indonesia Vs Timnas Filipina 2-0 Indone...,1800679270857286043,NaN,NaN,in,NaN,0,0,0,https://x.com/kangsil2012/status/1800679270857...,546890263,kangsil2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996,1799468917662126553,Sat Jun 08 15:49:57 +0000 2024,0,Tapi ketika Timnas Junior selalu berganti pela...,1799468919666917547,NaN,fltysxh,in,lndonesia,0,0,0,https://x.com/fltysxh/status/1799468919666917547,619677725,fltysxh
1997,1799468756919542104,Sat Jun 08 15:49:18 +0000 2024,0,ciwi ciwi yg kemaren pd nonton bola boleh bawa...,1799468756919542104,NaN,NaN,in,NaN,0,0,0,https://x.com/jellybejay/status/17994687569195...,1390998168679051270,jellybejay
1998,1799440783411683804,Sat Jun 08 15:49:18 +0000 2024,0,@Nicoadic @idextratime @bliblidotcom Sedangkan...,1799468755262824698,NaN,Nicoadic,in,NaN,0,0,0,https://x.com/Silentisgold551/status/179946875...,1672528275854479360,Silentisgold551
1999,1799468669938155766,Sat Jun 08 15:48:58 +0000 2024,123,Gw baru ngeh ternyata ada yang gak dipanggil T...,1799468669938155766,NaN,NaN,in,Stamford Bridge,5,28,4,https://x.com/KABAR_CHELSEA/status/17994686699...,339643428,KABAR_CHELSEA


## 1. Preposessing

In [216]:
import pandas as pd
import re
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory


# Create stopword list
factory = StopWordRemoverFactory()
stop_words = set(factory.get_stop_words())

# Tambahkan kata hubung tambahan ke dalam daftar stopwords
kata_hubung = {'dan', 'atau', 'tetapi', 'namun', 'sehingga', 'karena', 'jika', 'jikalau', 'maka', 'sebab', 'meskipun', 'walaupun', 'yg','lu','gue','gini','klo','sama'}
stop_words.update(kata_hubung)

# Function to preprocess tweets
def preprocess_tweet(tweet):
    tweet = re.sub(r'http\S+', '', tweet)  # Remove URLs
    tweet = re.sub(r'@\w+', '', tweet)  # Remove mentions
    tweet = re.sub(r'#\w+', '', tweet)  # Remove hashtags
    tweet = re.sub(r'\d+', '', tweet)  # Remove numbers
    tweet = tweet.lower()  # Convert to lowercase
    tweet = re.sub(r'[^\w\s]', '', tweet)  # Remove punctuation
    tweet = tweet.split()  # Split into words
    tweet = [word for word in tweet if word not in stop_words]  # Remove stopwords
    return ' '.join(tweet)

# Apply preprocessing to each tweet
data['cleaned_tweet'] = data['full_text'].apply(preprocess_tweet)


## 2. Topic Modeling dengan LDA

In [217]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Vectorize the cleaned tweets
vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words=list(stop_words))
tweet_matrix = vectorizer.fit_transform(data['cleaned_tweet'])

# Fit the LDA model
lda = LatentDirichletAllocation(n_components=10, random_state=42)
lda.fit(tweet_matrix)

# Display the top words for each topic
def display_topics(model, feature_names, num_top_words):
    topics = []
    for topic_idx, topic in enumerate(model.components_):
        topics.append([feature_names[i] for i in topic.argsort()[:-num_top_words - 1:-1]])
    return topics

num_top_words = 10
feature_names = vectorizer.get_feature_names_out()
topics = display_topics(lda, feature_names, num_top_words)

for idx, topic in enumerate(topics):
    print(f"Topik {idx+1}: {', '.join(topic)}")


Topik 1: pemain, aja, indonesia, jangan, kalau, liga, tim, punya, klub, jadi
Topik 2: indonesia, piala, filipina, dunia, kualifikasi, lolos, vs, asia, laga, putaran
Topik 3: menang, indonesia, tiket, pemain, semoga, verdonk, filipina, kalo, garuda, wtb
Topik 4: main, ga, pemain, nonton, buat, kan, lebih, banget, kalo, aja
Topik 5: aja, pemain, ga, menang, nonton, jadi, kalah, dah, mau, main
Topik 6: nonton, pemain, gak, liat, hari, vidio, buat, ga, semua, kbulan
Topik 7: bgt, jadi, sampe, buat, jd, nonton, suka, inggris, gak, gw
Topik 8: indonesia, putri, filipina, hasil, bahrain, menang, lawan, apa, live, day
Topik 9: indonesia, nya, gbk, stadion, buat, bukan, jadi, bagus, pak, pemain
Topik 10: ga, aja, kalo, udah, fomo, jadi, gak, pemain, mau, nya


## 3. Clustering

In [218]:
import numpy as np

# Get topic distribution for each tweet
topic_distribution = lda.transform(tweet_matrix)

# Assign each tweet to the topic with the highest probability
data['topic_cluster'] = np.argmax(topic_distribution, axis=1)

# Save the DataFrame to a CSV file
data.to_csv('tweets_with_topics.csv', index=False)

# Display the first few rows with assigned topics
print(data[['full_text', 'topic_cluster']].head())


                                           full_text  topic_cluster
0  TIMNAS PRIA DAN WANITA MENANG + ERIK TEN HAG STAY              7
1  ngerasa ga sih sekarang timnas kalo main pakek...              3
2  2 Sejarah Dicetak Timnas Indonesia Usai Ganyan...              1
3  @womensfootie_id Tinggal tmbah pemain diaspora...              4
4  Timnas Indonesia Vs Timnas Filipina 2-0 Indone...              1


## 4. Interpretasi

In [219]:
data = data[['full_text', 'topic_cluster']]

data = data.sort_values(by='topic_cluster').reset_index(drop=True)

In [220]:
data.to_csv('tweet+topics.csv', index=False)

In [221]:
data

,full_text,topic_cluster
0,@shxxars131 @clordtod @gatauahdark @idextratim...,0
1,@kyentut Kasian timnas,0
2,memang euro sebaiknya dukung timnas pusat ajah...,0
3,Pelatih Filipina Tom Saintfiet: ️ Level Timnas...,0
4,Timnas Indonesia dipuji habis oleh pelatih Fil...,0
...,...,...
1996,Kalo karena fomo perempuan demand nonton timna...,9
1997,Gue against orang2 yang anti dengan masuknya s...,9
1998,@NoAbsen13 @MurdiYasin @idextratime @bliblidot...,9
1999,Btw tl aku isinya bukan lagi nct tapi nathan t...,9


Topik 1: Topik pertama mencakup diskusi tentang sejarah yang dicetak oleh Timnas Indonesia dan pertandingan melawan Filipina di Piala Dunia.

Topik 2: Topik kedua membahas mengenai Indonesia di Piala Dunia, pertandingan melawan Filipina, kualifikasi, dan kelolosan.

Topik 3: Topik ketiga fokus pada perasaan tentang permainan Timnas saat ini, bagaimana mereka bermain, dan harapan terhadap performa mereka.

Topik 4: Topik keempat mencakup saran untuk menambahkan pemain diaspora, diskusi tentang pemain dan strategi yang digunakan.

Topik 5: Topik kelima membahas mengenai fans yang merasa fomo (fear of missing out) terkait dengan Timnas, kemenangan, dan performa mereka.

Topik 6: Topik keenam menyoroti diskusi tentang pelatih Timnas dan strategi yang harus digunakan.

Topik 7: Topik ketujuh mencakup topik tentang pertandingan antara Timnas Indonesia dan Filipina serta kemenangan yang diraih.

Topik 8: Topik kedelapan membahas tentang kemarahan supporter ke Anang dan asyanti pada saat walkout.

Topik 9: Topik kesembilan fokus pada pertandingan U-19 Timnas Indonesia, perjalanan mereka ke Piala Asia U-19, dan performa mereka.

Topik 10: Topik kesepuluh mencakup kemenangan Timnas U-23 Indonesia dan performa impresif mereka di pertandingan.